<a href="https://colab.research.google.com/github/AnthonyPerezGit/DatingApp-V1.0/blob/master/Roof_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 🛰️ Real Roof Detection AI Training Notebook (Fully Fixed Version)
# Author: Your AI Partner 🚀

# ============================================
# STEP 0: Install Core Dependencies

!pip install -U "huggingface_hub[cli]" rasterio shapely geopandas tqdm --quiet

import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
import cv2
import rasterio
import geopandas as gpd
from shapely.geometry import shape
from tqdm import tqdm

print("✅ Libraries Installed!")

# ============================================
# STEP 1: Download SpaceNet Vegas Dataset (Direct Link)

print("📦 Downloading SpaceNet Vegas Dataset...")

!wget -O SN2_buildings_train_AOI_2_Vegas.tar.gz "https://spacenet-dataset.s3.amazonaws.com/spacenet/SN2_buildings/tarballs/SN2_buildings_train_AOI_2_Vegas.tar.gz"
!mkdir -p spacenet_vegas
!tar -xzf SN2_buildings_train_AOI_2_Vegas.tar.gz -C spacenet_vegas

print("✅ SpaceNet Vegas dataset downloaded and extracted!")

# ============================================
# STEP 2: Generate Images and Masks

IMAGE_DIR = "/content/spacenet_vegas/AOI_2_Vegas/PS-RGB/"
LABEL_DIR = "/content/spacenet_vegas/AOI_2_Vegas/geojson/buildings/train/"
OUTPUT_DIR = "/content/real_roof_dataset/"

os.makedirs(OUTPUT_DIR + "images", exist_ok=True)
os.makedirs(OUTPUT_DIR + "masks", exist_ok=True)

# Helper to rasterize polygons into mask
def rasterize_shapes(shapes, out_shape):
    return rasterio.features.rasterize(
        ((geom, 1) for geom in shapes),
        out_shape=out_shape,
        fill=0,
        dtype='uint8'
    )

print("🛠 Generating masks from polygons...")

image_paths = sorted(glob.glob(IMAGE_DIR + "*.tif"))
for img_path in tqdm(image_paths[:100]):  # Limit to 100 samples
    base = os.path.basename(img_path).replace(".tif", "")
    mask_path = LABEL_DIR + base + ".geojson"

    if not os.path.exists(mask_path):
        continue

    # Load image
    img = rasterio.open(img_path).read([1, 2, 3])
    img = np.transpose(img, (1, 2, 0))
    img = np.clip(img, 0, 255).astype(np.uint8)

    # Load polygons
    gdf = gpd.read_file(mask_path)
    polygons = gdf.geometry.values

    # Create mask
    mask = rasterize_shapes(polygons, out_shape=img.shape[:2])

    # Save JPG + PNG
    img_save_path = os.path.join(OUTPUT_DIR, "images", base + ".jpg")
    mask_save_path = os.path.join(OUTPUT_DIR, "masks", base + "_mask.png")

    cv2.imwrite(img_save_path, cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
    cv2.imwrite(mask_save_path, mask * 255)

print("✅ Image and mask generation complete!")

# ============================================
# STEP 3: Load Dataset

IMG_SIZE = 128

print("🛠 Loading dataset...")

def load_data(img_dir, mask_dir, img_size=IMG_SIZE):
    img_paths = sorted(glob.glob(os.path.join(img_dir, "*.jpg")))
    images = []
    masks = []

    for img_path in img_paths:
        base = os.path.basename(img_path).replace('.jpg', '')
        mask_path = os.path.join(mask_dir, base + '_mask.png')

        if not os.path.exists(mask_path):
            continue

        img = cv2.imread(img_path)
        img = cv2.resize(img, (img_size, img_size))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        images.append(img / 255.0)

        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        mask = cv2.resize(mask, (img_size, img_size))
        mask = np.expand_dims(mask, axis=-1) / 255.0
        masks.append(mask)

    return np.array(images), np.array(masks)

images, masks = load_data(OUTPUT_DIR + "images", OUTPUT_DIR + "masks")

print(f"✅ Loaded {len(images)} real roof image-mask pairs!")

# ============================================
# STEP 4: Train U-Net Model

X_train, X_val, y_train, y_val = train_test_split(images, masks, test_size=0.2, random_state=42)

print(f"✅ Training set: {len(X_train)} samples, Validation set: {len(X_val)} samples")

# Build U-Net

def unet_model(input_size=(IMG_SIZE, IMG_SIZE, 3)):
    inputs = tf.keras.Input(input_size)

    c1 = layers.Conv2D(16, (3, 3), activation="relu", padding="same")(inputs)
    c1 = layers.Conv2D(16, (3, 3), activation="relu", padding="same")(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)

    c2 = layers.Conv2D(32, (3, 3), activation="relu", padding="same")(p1)
    c2 = layers.Conv2D(32, (3, 3), activation="relu", padding="same")(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)

    c3 = layers.Conv2D(64, (3, 3), activation="relu", padding="same")(p2)
    c3 = layers.Conv2D(64, (3, 3), activation="relu", padding="same")(c3)
    p3 = layers.MaxPooling2D((2, 2))(c3)

    c4 = layers.Conv2D(128, (3, 3), activation="relu", padding="same")(p3)
    c4 = layers.Conv2D(128, (3, 3), activation="relu", padding="same")(c4)

    u5 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding="same")(c4)
    u5 = layers.concatenate([u5, c3])
    c5 = layers.Conv2D(64, (3, 3), activation="relu", padding="same")(u5)
    c5 = layers.Conv2D(64, (3, 3), activation="relu", padding="same")(c5)

    u6 = layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding="same")(c5)
    u6 = layers.concatenate([u6, c2])
    c6 = layers.Conv2D(32, (3, 3), activation="relu", padding="same")(u6)
    c6 = layers.Conv2D(32, (3, 3), activation="relu", padding="same")(c6)

    u7 = layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding="same")(c6)
    u7 = layers.concatenate([u7, c1])
    c7 = layers.Conv2D(16, (3, 3), activation="relu", padding="same")(u7)
    c7 = layers.Conv2D(16, (3, 3), activation="relu", padding="same")(c7)

    outputs = layers.Conv2D(1, (1, 1), activation="sigmoid")(c7)

    model = models.Model(inputs=[inputs], outputs=[outputs])
    return model

print("🛠 Building U-Net model...")

model = unet_model()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

# ============================================
# STEP 5: Train Model

print("🏋️ Training model...")

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    batch_size=4,
    epochs=20
)

# ============================================
# STEP 6: Plot Training Curves

plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title('Loss Curves')
plt.show()

# ============================================
# STEP 7: Save Model

model.save('roof_spacenet_model.h5')
print("✅ Model saved as roof_spacenet_model.h5!")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 83.3 MB/s eta 0:00:00
✅ Libraries Installed!
📦 Downloading SpaceNet Vegas Dataset...
--2025-04-21 02:58:47--  https://spacenet-dataset.s3.amazonaws.com/spacenet/SN2_buildings/tarballs/SN2_buildings_train_AOI_2_Vegas.tar.gz
Resolving spacenet-dataset.s3.amazonaws.com (spacenet-dataset.s3.amazonaws.com)... 3.5.0.10, 3.5.25.33, 16.15.217.244, ...
Connecting to spacenet-dataset.s3.amazonaws.com (spacenet-dataset.s3.amazonaws.com)|3.5.0.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25640007327 (24G) [application/x-tar]
Saving to: ‘SN2_buildings_train_AOI_2_Vegas.tar.gz’

SN2_buildings_train 100%[===================>]  23.88G  47.5MB/s    in 8m 21s  

2025-04-21 03:07:09 (48.8 MB/s) - ‘SN2_buildings_train_AOI_2_Vegas.tar.gz’ saved [25640007327/25640007327]

